# Middleware Project: Big Data

## Initialization

In [1]:
DATASET_DIR = './dataset/'

In [2]:
# imports
import operator
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from time import time

In [3]:
# initialize spark
#sc = pyspark.SparkContext("local[4]", "MiddlewareProject")  # 4 threads
spark = SparkSession.builder.appName("MiddlewareProject").getOrCreate()

In [4]:
# https://spark.apache.org/docs/latest/rdd-programming-guide.html

In [5]:
# open files

years = range(1994, 2009)
# years = range(2004, 2005)
path = DATASET_DIR + '{' + ','.join(str(y) for y in years) + '}.csv'
print('Loading from:', path)

# load entire file
df = spark.read.csv(path, header=True)
print('Number of entries:', df.count())

Loading from: ./dataset/{1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008}.csv
Number of entries: 91469371


In [6]:
# replace 'NA' with null values
df = df.replace('NA', None)

In [7]:
# cast columns to correct type
intcolumns = ['Year', 'Month', 'DayofMonth', 'DayofWeek', 'DepTime', 'CRSDEPTime', 'ArrTime', 'CRSArrTime',
              'ActualElapsedTime', 'CRSElapsedTime', 'AirTime', 'ArrDelay', 'DepDelay', 'Distance', 'CarrierDelay',
              'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay', 'Cancelled', 'Diverted']

for c in intcolumns:
    df = df.withColumn(c, df[c].cast('int'))
df.first()

Row(Year=2002, Month=1, DayofMonth=13, DayofWeek=7, DepTime=2231, CRSDEPTime=2235, ArrTime=2342, CRSArrTime=2353, UniqueCarrier='US', FlightNum='723', TailNum='N709��', ActualElapsedTime=71, CRSElapsedTime=78, AirTime=55, ArrDelay=-11, DepDelay=-4, Origin='PIT', Dest='CLT', Distance=366, TaxiIn='3', TaxiOut='13', Cancelled=0, CancellationCode=None, Diverted=0, CarrierDelay=None, WeatherDelay=None, NASDelay=None, SecurityDelay=None, LateAircraftDelay=None)

### Percentage of cancelled flights per day

Average of percentage of cancelled flights of each day

In [8]:
# get RDD
raw_data = df.rdd

# raw_data.first()

In [9]:
#total flight per day
indexed_by_day = raw_data.map(lambda m: ((m['Year'], m['Month'], m['DayofMonth']),(m['Cancelled'],1)))
# indexed_by_day.take(10)


In [10]:
num_flights_day = indexed_by_day.reduceByKey(lambda a,b: (a[0] + b[0], a[1] + b[1]))
# num_flights_day.take(20)

In [11]:
#compute percentage per day
daily_percentage = num_flights_day.map(lambda m: m[1][0]*100.0/m[1][1])

In [12]:
# daily_percentage.take(10)

In [13]:
#compute average

# execute query
starttime = time()
res = daily_percentage.reduce(lambda a,b: a+b)/daily_percentage.count()
endtime = time() - starttime
print('Time taken: {:.2f} seconds'.format(endtime))

print("Percentage of cancelled flights per day: ", res)

Time taken: 872.66 seconds
Percentage of cancelled flights per day:  2.127927917735455


### Penalty scores for each airport

A weekly "penalty" score for each airport that depends on both its incoming and outgoing flights. 
The score adds 
- 0.5 for each incoming flight that is more than 15 minutes late
- 1 for each outgoing flight that is more than 15 minutes late

In [14]:
raw_data = df.rdd

In [15]:
# Keep only records with relevant delays
airports_with_delays = raw_data.filter(lambda f: ((f['ArrDelay'] is not None and f['ArrDelay'] > 15) or (f['DepDelay'] is not None and f['DepDelay'] > 15)))

# airports_with_delays.count()

In [16]:
from datetime import datetime

# add the week as key
def find_week(row):
    '''Returns the week as a tuple (year, week in the year)'''
    d = datetime(year=row['Year'], month=row['Month'], day=row['DayofMonth'])
    ic = d.isocalendar()
    return (ic[0], ic[1])

indexed_by_week = airports_with_delays.map(lambda r: (find_week(r), r))

In [17]:
def dep_penalty(row):
    res = ()
    if(row['DepDelay'] is not None and row['DepDelay'] > 15):
        res =  (row['Origin'], 1)
    return res
    
def arr_penalty(row):
    res = ()
    if(row['ArrDelay'] is not None and row['ArrDelay'] > 15):
        res = (row['Dest'], 0.5)
    return res

In [18]:
final = indexed_by_week.flatMap(lambda m: ((m[0],dep_penalty(m[1])),(m[0],arr_penalty(m[1]))))
final = final.filter(lambda f: f[1] is not ())
final = final.map(lambda m: ((m[0][0],m[0][1],m[1][0]),(m[1][1])))
final = final.reduceByKey(lambda a,b:( a+b))

In [19]:
# execute query
starttime = time()
weekly_pen = final.collect()
endtime = time() - starttime
print('Time taken: {:.2f} seconds'.format(endtime))

Time taken: 985.93 seconds


In [20]:
# sort by week
weekly_pen = sorted(weekly_pen, key=lambda x: x[0])
weekly_pen

[((1993, 52, 'ABE'), 6.0),
 ((1993, 52, 'ABQ'), 17.0),
 ((1993, 52, 'AGS'), 5.5),
 ((1993, 52, 'ALB'), 12.0),
 ((1993, 52, 'AMA'), 4.5),
 ((1993, 52, 'ANC'), 17.5),
 ((1993, 52, 'ATL'), 377.5),
 ((1993, 52, 'AUS'), 24.5),
 ((1993, 52, 'AVL'), 2),
 ((1993, 52, 'AVP'), 5.0),
 ((1993, 52, 'AZO'), 1.0),
 ((1993, 52, 'BDL'), 26.0),
 ((1993, 52, 'BFL'), 1),
 ((1993, 52, 'BGM'), 2.5),
 ((1993, 52, 'BGR'), 0.5),
 ((1993, 52, 'BHM'), 16.0),
 ((1993, 52, 'BIL'), 5.0),
 ((1993, 52, 'BIS'), 1.5),
 ((1993, 52, 'BNA'), 32.0),
 ((1993, 52, 'BOI'), 9.0),
 ((1993, 52, 'BOS'), 92.5),
 ((1993, 52, 'BTM'), 0.5),
 ((1993, 52, 'BTR'), 2.0),
 ((1993, 52, 'BTV'), 2.5),
 ((1993, 52, 'BUF'), 26.5),
 ((1993, 52, 'BUR'), 22.0),
 ((1993, 52, 'BWI'), 98.0),
 ((1993, 52, 'BZN'), 3.0),
 ((1993, 52, 'CAE'), 5.0),
 ((1993, 52, 'CHA'), 5.0),
 ((1993, 52, 'CHS'), 14.5),
 ((1993, 52, 'CID'), 4.5),
 ((1993, 52, 'CLE'), 53.5),
 ((1993, 52, 'CLT'), 172.5),
 ((1993, 52, 'CMH'), 70.0),
 ((1993, 52, 'COS'), 5.0),
 ((1993, 52, '

In [21]:
# sort by airport
sorted(weekly_pen, key=lambda x: x[0][2])

[((1993, 52, 'ABE'), 6.0),
 ((1994, 1, 'ABE'), 36.0),
 ((1994, 2, 'ABE'), 26.5),
 ((1994, 3, 'ABE'), 36.5),
 ((1994, 4, 'ABE'), 43.0),
 ((1994, 5, 'ABE'), 16.5),
 ((1994, 6, 'ABE'), 31.0),
 ((1994, 7, 'ABE'), 18.0),
 ((1994, 8, 'ABE'), 43.5),
 ((1994, 9, 'ABE'), 21.5),
 ((1994, 10, 'ABE'), 16.0),
 ((1994, 11, 'ABE'), 16.0),
 ((1994, 12, 'ABE'), 15.5),
 ((1994, 13, 'ABE'), 7.0),
 ((1994, 14, 'ABE'), 10.0),
 ((1994, 15, 'ABE'), 23.0),
 ((1994, 16, 'ABE'), 1.5),
 ((1994, 17, 'ABE'), 19.0),
 ((1994, 18, 'ABE'), 7.5),
 ((1994, 19, 'ABE'), 14.0),
 ((1994, 20, 'ABE'), 14.5),
 ((1994, 21, 'ABE'), 13.5),
 ((1994, 22, 'ABE'), 7.0),
 ((1994, 23, 'ABE'), 19.0),
 ((1994, 24, 'ABE'), 26.5),
 ((1994, 25, 'ABE'), 28.0),
 ((1994, 26, 'ABE'), 34.5),
 ((1994, 27, 'ABE'), 22.0),
 ((1994, 28, 'ABE'), 27.5),
 ((1994, 29, 'ABE'), 36.5),
 ((1994, 30, 'ABE'), 20.0),
 ((1994, 31, 'ABE'), 30.0),
 ((1994, 32, 'ABE'), 14.0),
 ((1994, 33, 'ABE'), 20.5),
 ((1994, 34, 'ABE'), 18.0),
 ((1994, 35, 'ABE'), 10.0),
 ((199

### Our group's data analysis

In [22]:
# an additional data analysis defined by your group

# IDEE
# qualcosa con gli altri delays? security, nas, carrier
# qualche confronto tra carrier
# 

In [23]:
# prima fare con un solo file come debug, poi mettere tutti